In [1]:
import torch
import torch.nn as nn
from model import Transformer
from train import get_model, get_ds, greedy_decode
import altair as alt
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [3]:
from config import get_config, get_weights_file_path
config = get_config()
train_dataloader, val_dataloader, vocab_src, vocab_tgt = get_ds(config)
model = get_model(config, vocab_src.get_vocab_size(), vocab_tgt.get_vocab_size()).to(device)

# Load the pretrained weights
# model_filename = get_weights_file_path(config, f"04")
# model_filename = './weights/L1H6D36maxPos200_99.pt'
state = torch.load(model_filename)
model.load_state_dict(state['model_state_dict'])

Max length of source sentence: 100
Max length of target sentence: 100


RuntimeError: Error(s) in loading state_dict for Transformer:
	Missing key(s) in state_dict: "encoder.layers.1.self_attention_block.w_q.weight", "encoder.layers.1.self_attention_block.w_k.weight", "encoder.layers.1.self_attention_block.w_v.weight", "encoder.layers.1.self_attention_block.w_o.weight", "encoder.layers.1.feed_forward_block.linear_1.weight", "encoder.layers.1.feed_forward_block.linear_1.bias", "encoder.layers.1.feed_forward_block.linear_2.weight", "encoder.layers.1.feed_forward_block.linear_2.bias", "encoder.layers.1.residual_connections.0.norm.alpha", "encoder.layers.1.residual_connections.0.norm.bias", "encoder.layers.1.residual_connections.1.norm.alpha", "encoder.layers.1.residual_connections.1.norm.bias", "encoder.layers.2.self_attention_block.w_q.weight", "encoder.layers.2.self_attention_block.w_k.weight", "encoder.layers.2.self_attention_block.w_v.weight", "encoder.layers.2.self_attention_block.w_o.weight", "encoder.layers.2.feed_forward_block.linear_1.weight", "encod

In [ ]:
# plot image    
import matplotlib.pyplot as plt
%matplotlib inline

ss = model.decoder.layers[0].cross_attention_block.attention_scores
ss = ss[0][0].detach().cpu().numpy()
# Plot the attention scores
for h in range(1, config['num_heads']):
    s = model.decoder.layers[0].cross_attention_block.attention_scores
    s = s[0][h].detach().cpu().numpy()
    ss = ss + s
    ss = ss / config['num_heads']
    plt.figure(figsize=(4,4))
    plt.imshow(s, cmap='coolwarm', interpolation='nearest')
    plt.colorbar()
    plt.show()
plt.close()


AttributeError: 'MultiHeadAttentionBlock' object has no attribute 'attention_scores'

In [ ]:
def load_next_batch():
    # Load a sample batch from the validation set
    batch = next(iter(val_dataloader))
    encoder_input = batch["encoder_input"].to(device)
    encoder_mask = batch["encoder_mask"].to(device)
    decoder_input = batch["decoder_input"].to(device)
    decoder_mask = batch["decoder_mask"].to(device)

    encoder_input_tokens = [vocab_src.id_to_token(idx) for idx in encoder_input[0].cpu().numpy()]
    decoder_input_tokens = [vocab_tgt.id_to_token(idx) for idx in decoder_input[0].cpu().numpy()]

    # check that the batch size is 1
    assert encoder_input.size(
        0) == 1, "Batch size must be 1 for validation"

    model_out = greedy_decode(
        model, encoder_input, encoder_mask, vocab_src, vocab_tgt, config['seq_len'], device)
    
    return batch, encoder_input_tokens, decoder_input_tokens

In [ ]:
def mtx2df(m, max_row, max_col, row_tokens, col_tokens):
    return pd.DataFrame(
        [
            (
                r,
                c,
                float(m[r, c]),
                "%.3d %s" % (r, row_tokens[r] if len(row_tokens) > r else "<blank>"),
                "%.3d %s" % (c, col_tokens[c] if len(col_tokens) > c else "<blank>"),
            )
            for r in range(m.shape[0])
            for c in range(m.shape[1])
            if r < max_row and c < max_col
        ],
        columns=["row", "column", "value", "row_token", "col_token"],
    )

def get_attn_map(attn_type: str, layer: int, head: int):
    if attn_type == "encoder":
        attn = model.encoder.layers[layer].self_attention_block.attention_scores
    elif attn_type == "decoder":
        attn = model.decoder.layers[layer].self_attention_block.attention_scores
    elif attn_type == "encoder-decoder":
        attn = model.decoder.layers[layer].cross_attention_block.attention_scores
    return attn[0, head].data

def attn_map(attn_type, layer, head, row_tokens, col_tokens, max_sentence_len):
    df = mtx2df(
        get_attn_map(attn_type, layer, head),
        max_sentence_len,
        max_sentence_len,
        row_tokens,
        col_tokens,
    )
    return (
        alt.Chart(data=df)
        .mark_rect()
        .encode(
            x=alt.X("col_token", axis=alt.Axis(title="")),
            y=alt.Y("row_token", axis=alt.Axis(title="")),
            color="value",
            tooltip=["row", "column", "value", "row_token", "col_token"],
        )
        #.title(f"Layer {layer} Head {head}")
        .properties(height=400, width=400, title=f"Layer {layer} Head {head}")
        .interactive()
    )

def get_all_attention_maps(attn_type: str, layers: list[int], heads: list[int], row_tokens: list, col_tokens, max_sentence_len: int):
    charts = []
    for layer in layers:
        rowCharts = []
        for head in heads:
            rowCharts.append(attn_map(attn_type, layer, head, row_tokens, col_tokens, max_sentence_len))
        charts.append(alt.hconcat(*rowCharts))
    return alt.vconcat(*charts)

In [ ]:
batch, encoder_input_tokens, decoder_input_tokens = load_next_batch()
print(f'Source: {batch["src_text"][0]}')
print(f'Target: {batch["tgt_text"][0]}')
try:
    sentence_len = encoder_input_tokens.index("[PAD]")
except:
    sentence_len = config['seq_len']

Source: G13 G71 G90 G37 G9 G6 G2 G81 G52 G26 G19 G73 G29 G11 G25 G78 G30 G72 G84 G15 G33 G1 G97 G22 G51 G66 G82 G36 G60 G67 G56 G77 G100 G91 G4 G55 G62 G61 G65 G57 G94 G70 G59 G92 G44 G58 G68 G64 G3 G87 G63 G12 G27 G35 G89 G8 G54 G39 G32 G14 G76 G21 G69 G38 G83 G40 G80 G23 G7 G28 G93 G98 G74 G34 G31 G48 G88 G24 G18 G16 G43 G5 G53 G20 G46 G95 G17 G85 G49 G47 G10 G79 G99 G41 G86 G45 G50 G96 G42 G75
Target: G13 G90 G2 G81 G26 G9 G6 G19 G15 G78 G71 G11 G37 G52 G29 G72 G66 G84 G22 G25 G82 G73 G33 G91 G56 G51 G1 G30 G97 G36 G60 G100 G67 G4 G77 G55 G65 G61 G58 G94 G62 G70 G89 G64 G27 G59 G44 G57 G68 G41 G3 G24 G34 G12 G83 G75 G86 G43 G79 G8 G63 G23 G18 G87 G42 G40 G74 G76 G32 G14 G39 G20 G54 G50 G80 G31 G47 G98 G5 G17 G88 G35 G48 G38 G21 G10 G53 G46 G92 G96 G69 G85 G7 G93 G99 G16 G49 G45 G28 G95


In [ ]:
layers = [ i for i in range(config['num_layers']) ]
heads = [ i for i in range(config['num_heads']) ]

# Encoder Self-Attention
get_all_attention_maps("encoder", layers, heads, encoder_input_tokens, encoder_input_tokens, min(20, sentence_len))


alt.VConcatChart(...)

In [ ]:
# Decoder Self-Attention
get_all_attention_maps("decoder", layers, heads, decoder_input_tokens, decoder_input_tokens, min(20, sentence_len))

alt.VConcatChart(...)

In [ ]:
# cross-attention
get_all_attention_maps("encoder-decoder", layers, heads, encoder_input_tokens, decoder_input_tokens, min(20, sentence_len))

alt.VConcatChart(...)